In [1]:
#importing necessary libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [3]:
#importing facebook data
facebook_data = pd.read_csv("Performance_Facebook.csv")
facebook_data.head()

,Date,Post ID,Network,Post Type,Content Type,Profile,Sent by,Link,Post,Linked Content,...,Video Removed from Playlists,Annotation Impressions,Annotation Clickable Impressions,Annotation Closable Impressions,Annotation Closes,Card Impressions,Card Teaser Impressions,Card Teaser Clicks,Poll Votes,Tags
0,12/17/2022 5:08 pm,253788558082460_5441020446025886,Facebook,Post,Video,Stanbic IBTC,Damilare Oyekanmi,https://www.facebook.com/253788558082460/posts...,"We celebrated recently with Novare, one of our...",https://www.facebook.com/StanbicIBTC/videos/53...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CIB Posts
1,5/4/2019 10:01 am,253788558082460_2001824979945467,Facebook,Post,Photo,Stanbic IBTC,,https://www.facebook.com/253788558082460/posts...,N5k can get you started today. Call 01 280 126...,https://www.facebook.com/StanbicIBTC/photos/a....,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6/2/2020 9:00 pm,253788558082460_2768996106561680,Facebook,Post,Photo,Stanbic IBTC,,https://www.facebook.com/StanbicIBTC/photos/a....,Still not sure whether to invest in the FGN Bo...,https://www.facebook.com/StanbicIBTC/photos/a....,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5/9/2022 10:37 am,253788558082460_4808157902645480,Facebook,Post,Photo,Stanbic IBTC,Damilare Oyekanmi,https://www.facebook.com/StanbicIBTC/photos/a....,Let nothing stop you from achieving your goals...,https://www.facebook.com/StanbicIBTC/photos/a....,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7/18/2021 10:00 am,253788558082460_3890983261029620,Facebook,Post,Photo,Stanbic IBTC,,https://www.facebook.com/StanbicIBTC/photos/a....,"If you were to pick one wish, what would it be...",https://www.facebook.com/StanbicIBTC/photos/a....,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#checking shape of facebook data
print(facebook_data.shape)
#checking number of missing values per column
print(facebook_data.isnull().sum())

(9803, 147)
Date                          0
Post ID                       0
Network                       0
Post Type                     0
Content Type                  0
                           ... 
Card Impressions           9803
Card Teaser Impressions    9803
Card Teaser Clicks         9803
Poll Votes                 9803
Tags                       8202
Length: 147, dtype: int64


looks like a number of columns have no data in them at all so its hard to input missing values
without a baseline whatsoever. So we'll delete those columns 


In [5]:
#calculate number of null columns in each row and delete those whose threshold is equal to the number of total rows
null_counts = facebook_data.isnull().sum()
threshold = 9803
columns_to_drop = null_counts[null_counts == threshold].index
facebook_data = facebook_data.drop(columns=columns_to_drop)

#print columns left
facebook_data.columns

Index(['Date', 'Post ID', 'Network', 'Post Type', 'Content Type', 'Profile',
       'Sent by', 'Link', 'Post', 'Linked Content',
       ...
       'Organic Video View Time (Seconds)', 'Paid Video View Time (Seconds)',
       'Unique Video Views', 'Unique Organic Video Views',
       'Unique Paid Video Views', 'Unique 10-Second Video Views',
       'Unique Full Video Views', 'Unique Organic 95% Video Views',
       'Unique Paid 95% Video Views', 'Tags'],
      dtype='object', length=101)

In [6]:
#analyse the missing values in the columns left
print(facebook_data.isnull().sum())

Date                                 0
Post ID                              0
Network                              0
Post Type                            0
Content Type                         0
                                  ... 
Unique 10-Second Video Views      8876
Unique Full Video Views           8876
Unique Organic 95% Video Views    8876
Unique Paid 95% Video Views       8881
Tags                              8202
Length: 101, dtype: int64


for this analysis we'll drop columns that are not of primary importance
and have missing values over 8000

In [7]:
#dropping null counts above 8000 as performed above
null_counts = facebook_data.isnull().sum()
threshold = 8000
columns_to_drop = null_counts[null_counts >= threshold].index
facebook_data = facebook_data.drop(columns = columns_to_drop)

#print number of null values in the columns
print(facebook_data.isnull().sum())


Date                                   0
Post ID                                0
Network                                0
Post Type                              0
Content Type                           0
Profile                                0
Sent by                                0
Link                                   0
Post                                 250
Linked Content                       341
Impressions                          910
Organic Impressions                  910
Viral Impressions                    910
Non-viral Impressions                910
Paid Impressions                     910
Fan Impressions                      910
Fan Organic Impressions              910
Fan Paid Impressions                 910
Non-fan Impressions                  910
Non-fan Organic Impressions          910
Non-fan Paid Impressions             910
Reach                                910
Organic Reach                        910
Viral Reach                          910
Non-viral Reach 

In [8]:
#dropping columns not pertinent to the analysis or with redudant data
facebook_data = facebook_data.drop(["Post ID", "Link", "Post Type","Profile","Linked Content",
                                    "Paid Impressions", "Fan Organic Impressions", "Fan Paid Impressions",
                                    "Non-fan Organic Impressions", "Non-fan Paid Impressions",
                                    "Paid Reach","Fan Paid Reach"], axis =1)
facebook_data.columns


Index(['Date', 'Network', 'Content Type', 'Sent by', 'Post', 'Impressions',
       'Organic Impressions', 'Viral Impressions', 'Non-viral Impressions',
       'Fan Impressions', 'Non-fan Impressions', 'Reach', 'Organic Reach',
       'Viral Reach', 'Non-viral Reach', 'Fan Reach',
       'Engagement Rate (per Impression)', 'Engagement Rate (per Reach)',
       'Engagements', 'Reactions', 'Likes', 'Love Reactions', 'Haha Reactions',
       'Wow Reactions', 'Sad Reactions', 'Angry Reactions', 'Comments',
       'Shares', 'Click-Through Rate', 'Other Post Clicks',
       'Post Clicks (All)', 'Post Photo View Clicks', 'Answers',
       'Negative Feedback', 'Engaged Users', 'Engaged Fans',
       'Users Talking About This', 'Unique Reactions', 'Unique Comments',
       'Unique Shares', 'Unique Answers', 'Unique Post Clicks',
       'Unique Post Photo View Clicks', 'Unique Other Post Clicks',
       'Unique Negative Feedback'],
      dtype='object')

In [9]:
#checking for variation in values for the columns
distinct_values = facebook_data.nunique()
distinct_values

Date                                9389
Network                                1
Content Type                           4
Sent by                                7
Post                                8897
Impressions                         5672
Organic Impressions                 5654
Viral Impressions                   2047
Non-viral Impressions               5479
Fan Impressions                     5471
Non-fan Impressions                 2319
Reach                               4333
Organic Reach                       4231
Viral Reach                         1459
Non-viral Reach                     3679
Fan Reach                           3559
Engagement Rate (per Impression)     933
Engagement Rate (per Reach)         1006
Engagements                          998
Reactions                            494
Likes                                471
Love Reactions                        86
Haha Reactions                        24
Wow Reactions                         13
Sad Reactions   

there is enough variability in the data to warrant analysis now to deal with the missing values 
in the columns left

In [10]:
facebook_data.isnull().sum()

Date                                   0
Network                                0
Content Type                           0
Sent by                                0
Post                                 250
Impressions                          910
Organic Impressions                  910
Viral Impressions                    910
Non-viral Impressions                910
Fan Impressions                      910
Non-fan Impressions                  910
Reach                                910
Organic Reach                        910
Viral Reach                          910
Non-viral Reach                      910
Fan Reach                            910
Engagement Rate (per Impression)    2059
Engagement Rate (per Reach)         2062
Engagements                          910
Reactions                            910
Likes                                910
Love Reactions                       910
Haha Reactions                       910
Wow Reactions                        910
Sad Reactions   

In [11]:
#dropping rows where the post is not contained in the dataset
facebook_data = facebook_data.dropna(subset = ["Post"])
facebook_data.isnull().sum()

Date                                   0
Network                                0
Content Type                           0
Sent by                                0
Post                                   0
Impressions                          810
Organic Impressions                  810
Viral Impressions                    810
Non-viral Impressions                810
Fan Impressions                      810
Non-fan Impressions                  810
Reach                                810
Organic Reach                        810
Viral Reach                          810
Non-viral Reach                      810
Fan Reach                            810
Engagement Rate (per Impression)    1939
Engagement Rate (per Reach)         1942
Engagements                          810
Reactions                            810
Likes                                810
Love Reactions                       810
Haha Reactions                       810
Wow Reactions                        810
Sad Reactions   

In [12]:
#analysing the posts which have null reach 
posts_with_null_reach  = facebook_data[facebook_data["Reach"].isnull()]
# Print the posts where "reach" is null
print(posts_with_null_reach['Post'])

8895    Let us go back in time to when the Stanbic IBT...
8896    The weekend is about to get lit 💃 💃\nIn two da...
8897    It’s finally here! The Reward4Saving 2.0 Grand...
8898    You have the freedom to choose with the Stanbi...
8899    New experience center alert! 💃🏾🕺🏾\nStanbic IBT...
                              ...                        
9793    Check out our new TV ad and tell us what you t...
9794    An account with Stanbic IBTC opens the doors t...
9795    We want to help you reach your dreams. Let us ...
9796    The "Banking on Social Media" session took pla...
9797    Dbanj performing at the 2013 Orange AFCON clos...
Name: Post, Length: 810, dtype: object



the posts seem to have no issue lets try to review based on impressions


In [13]:
#analyzing the posts with null impressions
posts_with_null_impressions = facebook_data[facebook_data["Impressions"].isnull()]
posts_with_null_impressions["Post"]

8895    Let us go back in time to when the Stanbic IBT...
8896    The weekend is about to get lit 💃 💃\nIn two da...
8897    It’s finally here! The Reward4Saving 2.0 Grand...
8898    You have the freedom to choose with the Stanbi...
8899    New experience center alert! 💃🏾🕺🏾\nStanbic IBT...
                              ...                        
9793    Check out our new TV ad and tell us what you t...
9794    An account with Stanbic IBTC opens the doors t...
9795    We want to help you reach your dreams. Let us ...
9796    The "Banking on Social Media" session took pla...
9797    Dbanj performing at the 2013 Orange AFCON clos...
Name: Post, Length: 810, dtype: object

In [14]:
#seems like there is a particluar batch of posts with no metrics so we'll drop those as well
facebook_data = facebook_data.dropna(subset = ["Reach"])
facebook_data.isnull().sum()

Date                                   0
Network                                0
Content Type                           0
Sent by                                0
Post                                   0
Impressions                            0
Organic Impressions                    0
Viral Impressions                      0
Non-viral Impressions                  0
Fan Impressions                        0
Non-fan Impressions                    0
Reach                                  0
Organic Reach                          0
Viral Reach                            0
Non-viral Reach                        0
Fan Reach                              0
Engagement Rate (per Impression)    1129
Engagement Rate (per Reach)         1132
Engagements                            0
Reactions                              0
Likes                                  0
Love Reactions                         0
Haha Reactions                         0
Wow Reactions                          0
Sad Reactions   

In [15]:
#checking data types for calculations that are likely to follow
facebook_data.dtypes

Date                                 object
Network                              object
Content Type                         object
Sent by                              object
Post                                 object
Impressions                          object
Organic Impressions                  object
Viral Impressions                    object
Non-viral Impressions                object
Fan Impressions                      object
Non-fan Impressions                  object
Reach                                object
Organic Reach                        object
Viral Reach                          object
Non-viral Reach                      object
Fan Reach                            object
Engagement Rate (per Impression)     object
Engagement Rate (per Reach)          object
Engagements                          object
Reactions                            object
Likes                                object
Love Reactions                      float64
Haha Reactions                  

In [16]:
# Replace thousand separators (,) and extra spaces with an empty string in the 'Impressions' and 'Engagements' columns
# this is meant to calculate the 'Engagement Rate (per Impression)'
facebook_data[['Impressions', 'Engagements']] = facebook_data[['Impressions', 'Engagements']].apply(lambda x: x.str.replace(r'[,\s]+', '', regex=True))

# Convert the 'Impressions' and 'Engagements' columns to numeric
facebook_data[['Impressions', 'Engagements']] = facebook_data[['Impressions', 'Engagements']].apply(pd.to_numeric, errors='coerce')

# Check the data types of the 'Engagements' and 'Impressions' columns
print(facebook_data[['Engagements', 'Impressions']].dtypes)


Engagements      int64
Impressions    float64
dtype: object


In [17]:
# Check the number of NaN values in the 'Engagements' and 'Impressions' columns after the conversion
nan_engagements = facebook_data['Engagements'].isna().sum()
nan_impressions = facebook_data['Impressions'].isna().sum()

print(f"Number of NaN values in 'Engagements': {nan_engagements}")
print(f"Number of NaN values in 'Impressions': {nan_impressions}")


Number of NaN values in 'Engagements': 0
Number of NaN values in 'Impressions': 678


In [18]:
#the NaN values represent zero impressions  which are a key metric so lets analyze them further
# Print rows where 'Impressions' column is equal to zero
facebook_data[facebook_data['Impressions'] == 0]

,Date,Network,Content Type,Sent by,Post,Impressions,Organic Impressions,Viral Impressions,Non-viral Impressions,Fan Impressions,...,Engaged Fans,Users Talking About This,Unique Reactions,Unique Comments,Unique Shares,Unique Answers,Unique Post Clicks,Unique Post Photo View Clicks,Unique Other Post Clicks,Unique Negative Feedback
7710,7/7/2023 4:27 pm,Facebook,Video,Damilare Oyekanmi,How do you pursue happiness? Your Stanbic IBTC...,0.0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0,NaN,0,0.0
7711,7/3/2023 11:59 am,Facebook,Photo,Aramide Salami,Driving growth beyond borders is what we do. N...,0.0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0,0.0,0,0.0
7712,6/7/2023 1:30 pm,Facebook,Photo,Kemi Amoo,Did you know that you can use your retirement ...,0.0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0,0.0,0,0.0
7713,5/16/2023 8:30 pm,Facebook,Photo,Kemi Amoo,We celebrate the boy child today and every day...,0.0,0,0,0,0,...,1,0,0,0.0,0.0,0.0,1,1.0,0,0.0
7714,5/4/2023 1:27 pm,Facebook,Video,Damilare Oyekanmi,Flooding your timelines with some amazing test...,0.0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0,NaN,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8184,11/21/2016 7:30 pm,Facebook,Photo,Dunni Ogunsanya,You can win invites to this year's edition of ...,0.0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0,0.0,0,0.0
8185,11/21/2016 5:52 pm,Facebook,Photo,Dunni Ogunsanya,Have you seen some of the art we’ve shown here...,0.0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0,0.0,0,0.0
8186,11/21/2016 2:00 pm,Facebook,Photo,,The pessimist sees difficulty in every opportu...,0.0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0,0.0,0,0.0
8187,11/21/2016 12:44 pm,Facebook,Photo,Dunni Ogunsanya,"Musicomedy sensation, Kenny Blaq will be light...",0.0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0,0.0,0,0.0


In [19]:
#seems like posts with NaN impressions do have zero metric counts for all the other fields 
#we are considering posts with atleast one impression for this analaysis so those will be dropped
# Drop rows where 'Impressions' is NaN
facebook_data = facebook_data.dropna(subset=['Impressions'])
facebook_data = facebook_data[facebook_data['Impressions'] != 0]
# Reset the index of the DataFrame
facebook_data.reset_index(drop=True, inplace=True)
facebook_data.isnull().sum()

Date                                   0
Network                                0
Content Type                           0
Sent by                                0
Post                                   0
Impressions                            0
Organic Impressions                    0
Viral Impressions                      0
Non-viral Impressions                  0
Fan Impressions                        0
Non-fan Impressions                    0
Reach                                  0
Organic Reach                          0
Viral Reach                            0
Non-viral Reach                        0
Fan Reach                              0
Engagement Rate (per Impression)       0
Engagement Rate (per Reach)            3
Engagements                            0
Reactions                              0
Likes                                  0
Love Reactions                         0
Haha Reactions                         0
Wow Reactions                          0
Sad Reactions   


dropping values with less than 1 impression seems to eliminate null values for most of the other metrics
will turn the rest into a zero as mainly its click data and an absence of a variable can signify abscence of clicks


In [20]:
#fill missing values with zero
facebook_data = facebook_data.fillna(0)
facebook_data.isnull().sum()

Date                                0
Network                             0
Content Type                        0
Sent by                             0
Post                                0
Impressions                         0
Organic Impressions                 0
Viral Impressions                   0
Non-viral Impressions               0
Fan Impressions                     0
Non-fan Impressions                 0
Reach                               0
Organic Reach                       0
Viral Reach                         0
Non-viral Reach                     0
Fan Reach                           0
Engagement Rate (per Impression)    0
Engagement Rate (per Reach)         0
Engagements                         0
Reactions                           0
Likes                               0
Love Reactions                      0
Haha Reactions                      0
Wow Reactions                       0
Sad Reactions                       0
Angry Reactions                     0
Comments    

In [21]:
facebook_data.head()

,Date,Network,Content Type,Sent by,Post,Impressions,Organic Impressions,Viral Impressions,Non-viral Impressions,Fan Impressions,...,Engaged Fans,Users Talking About This,Unique Reactions,Unique Comments,Unique Shares,Unique Answers,Unique Post Clicks,Unique Post Photo View Clicks,Unique Other Post Clicks,Unique Negative Feedback
0,12/17/2022 5:08 pm,Facebook,Video,Damilare Oyekanmi,"We celebrated recently with Novare, one of our...",207378.0,"207,378",5,"207,373","1,147",...,51,231,177,53.0,2.0,0.0,587,0,577,13.0
1,5/4/2019 10:01 am,Facebook,Photo,,N5k can get you started today. Call 01 280 126...,125784.0,"125,784","23,926","101,858","103,287",...,"3,360","1,166",762,413.0,47.0,0.0,"3,412",931,"2,896",10.0
2,6/2/2020 9:00 pm,Facebook,Photo,,Still not sure whether to invest in the FGN Bo...,89699.0,"89,699","9,319","80,380","79,873",...,"2,033",834,446,423.0,28.0,0.0,"1,948",713,"1,470",5.0
3,5/9/2022 10:37 am,Facebook,Photo,Damilare Oyekanmi,Let nothing stop you from achieving your goals...,79118.0,"79,118",170,"78,948","3,352",...,43,110,100,9.0,2.0,0.0,290,23,275,1.0
4,7/18/2021 10:00 am,Facebook,Photo,,"If you were to pick one wish, what would it be...",76759.0,"76,759","20,443","56,316","55,174",...,"2,261","1,061",404,797.0,9.0,0.0,"2,570",124,"2,528",3.0


In [22]:
#splitting the date and time columns
facebook_data['Datetime'] =pd.to_datetime(facebook_data['Date'], format='%m/%d/%Y %I:%M %p')
facebook_data['Date only'] = facebook_data['Datetime'].dt.date
facebook_data['Time'] = facebook_data['Datetime'].dt.time
facebook_data[['Date only', 'Time']]

,Date only,Time
0,2022-12-17,17:08:00
1,2019-05-04,10:01:00
2,2020-06-02,21:00:00
3,2022-05-09,10:37:00
4,2021-07-18,10:00:00
...,...,...
7593,2023-04-06,11:16:00
7594,2023-04-05,18:19:00
7595,2023-05-02,15:39:00
7596,2022-11-25,10:49:00


In [23]:
facebook_data.head(1)

,Date,Network,Content Type,Sent by,Post,Impressions,Organic Impressions,Viral Impressions,Non-viral Impressions,Fan Impressions,...,Unique Comments,Unique Shares,Unique Answers,Unique Post Clicks,Unique Post Photo View Clicks,Unique Other Post Clicks,Unique Negative Feedback,Datetime,Date only,Time
0,12/17/2022 5:08 pm,Facebook,Video,Damilare Oyekanmi,"We celebrated recently with Novare, one of our...",207378.0,"207,378",5,"207,373","1,147",...,53.0,2.0,0.0,587,0,577,13.0,2022-12-17 17:08:00,2022-12-17,17:08:00


In [24]:
# cleaning up the columns
facebook_data = facebook_data.drop(columns=['Date','Datetime'])
facebook_data = facebook_data.rename(columns={'Date only': 'Date'})
facebook_data.head(1)

,Network,Content Type,Sent by,Post,Impressions,Organic Impressions,Viral Impressions,Non-viral Impressions,Fan Impressions,Non-fan Impressions,...,Unique Reactions,Unique Comments,Unique Shares,Unique Answers,Unique Post Clicks,Unique Post Photo View Clicks,Unique Other Post Clicks,Unique Negative Feedback,Date,Time
0,Facebook,Video,Damilare Oyekanmi,"We celebrated recently with Novare, one of our...",207378.0,"207,378",5,"207,373","1,147","206,231",...,177,53.0,2.0,0.0,587,0,577,13.0,2022-12-17,17:08:00


In [25]:
#saving the file as a csv 
facebook_data.to_csv('facebook_data.csv', index=False)


cleaning instagram data


In [26]:
#importing instagram data
instagram_data = pd.read_csv('Performance_Instagram.csv')
instagram_data.head()

,Date,Post ID,Network,Post Type,Content Type,Profile,Sent by,Link,Post,Linked Content,...,Video Removed from Playlists,Annotation Impressions,Annotation Clickable Impressions,Annotation Closable Impressions,Annotation Closes,Card Impressions,Card Teaser Impressions,Card Teaser Clicks,Poll Votes,Tags
0,1/30/2023 5:06 pm,18216468940204235,Instagram,Post,Carousel,stanbicibtc,,https://www.instagram.com/p/CoC6CKBIHbV/,The new Naira banknotes in circulation have se...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4/15/2022 9:04 am,17971531720518163,Instagram,Post,Photo,stanbicibtc,Blessing Ubah,https://www.instagram.com/p/CcXUQTEMRm7/,We hope the faith that this #GoodFriday restor...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5/28/2021 8:45 pm,18008486986321009,Instagram,Post,Video,stanbicibtc,,https://www.instagram.com/p/CPbcdjygiaK/,Reposted from @mzdosunmu My son's entry\n#ITCA...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1/4/2023 4:08 pm,17903404469640387,Instagram,Post,Carousel,stanbicibtc,,https://www.instagram.com/p/Cm_2oino-Me/,The new Naira banknotes in circulation have se...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12/7/2021 6:52 pm,17950858072558814,Instagram,Post,Photo,stanbicibtc,Blessing Ubah,https://www.instagram.com/p/CXMNAKFsjSs/,"If you were born in June, ‘cut soap for us oh’...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
#explore shape of the data and the missing values
print(instagram_data.shape)
instagram_data.isnull().sum()

(10000, 147)


Date                           0
Post ID                        0
Network                        0
Post Type                      0
Content Type                   0
                           ...  
Card Impressions           10000
Card Teaser Impressions    10000
Card Teaser Clicks         10000
Poll Votes                 10000
Tags                        8454
Length: 147, dtype: int64

In [28]:
#deleting columns with no data at all
threshold = 10000
null_count = instagram_data.isnull().sum()
columns_to_drop = null_count[null_count == threshold].index
instagram_data = instagram_data.drop(columns = columns_to_drop)
instagram_data.isnull().sum()

Date                                   0
Post ID                                0
Network                                0
Post Type                              0
Content Type                           0
Profile                                0
Sent by                                0
Link                                   0
Post                                  11
Impressions                         1484
Organic Impressions                 1484
Reach                               1484
Organic Reach                       1484
Engagement Rate (per Impression)    1768
Engagement Rate (per Reach)         2747
Engagements                            0
Reactions                              0
Likes                                  0
Comments                               0
Saves                               1484
Video Views                         8645
Organic Video Views                 8645
Tags                                8454
dtype: int64

In [29]:
#dropping columns with more than 8000 missing values
instagram_data = instagram_data.drop(columns = ['Video Views','Organic Video Views','Tags'])
#dropping unnecessary columns
instagram_data = instagram_data.drop(columns = ['Post ID', 'Link','Profile','Post Type'])
#dropping rows with null posts
instagram_data = instagram_data.dropna(subset = 'Post')
instagram_data.isnull().sum()                                     

Date                                   0
Network                                0
Content Type                           0
Sent by                                0
Post                                   0
Impressions                         1482
Organic Impressions                 1482
Reach                               1482
Organic Reach                       1482
Engagement Rate (per Impression)    1766
Engagement Rate (per Reach)         2745
Engagements                            0
Reactions                              0
Likes                                  0
Comments                               0
Saves                               1482
dtype: int64

In [30]:
#analysing posts with zero reach
posts_with_null_reach = instagram_data[instagram_data['Reach'].isnull()]
posts_with_null_reach['Post']

8516    At Stanbic IBTC, we hold special gatherings fo...
8517    Motor insurance is required for any ‘Idan’ who...
8518    Did you know that you can use your retirement ...
8519    We celebrate the boy child today and every day...
8520    Make your money bloom with the right investmen...
                              ...                        
9995    Withdraw cash up to 5 times, pay bills or tran...
9996    #CardDiaries week 13\n\nI love Christmas! In m...
9997    Eat-in this holidays. \nInstead of going out t...
9998    If you do not move forward, you will always be...
9999    Travel to Dubai this season with #StanbicDelig...
Name: Post, Length: 1482, dtype: object

In [31]:
#checking if posts with null reach have any other metrics
posts_with_null_reach

,Date,Network,Content Type,Sent by,Post,Impressions,Organic Impressions,Reach,Organic Reach,Engagement Rate (per Impression),Engagement Rate (per Reach),Engagements,Reactions,Likes,Comments,Saves
8516,6/12/2023 1:08 pm,Instagram,Carousel,Blessing Ubah,"At Stanbic IBTC, we hold special gatherings fo...",NaN,NaN,NaN,NaN,NaN,NaN,5,5,5,0,NaN
8517,6/8/2023 9:55 am,Instagram,Photo,Blessing Ubah,Motor insurance is required for any ‘Idan’ who...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN
8518,6/7/2023 1:30 pm,Instagram,Photo,Kemi Amoo,Did you know that you can use your retirement ...,NaN,NaN,NaN,NaN,NaN,NaN,8,4,4,4,NaN
8519,5/16/2023 8:30 pm,Instagram,Photo,Kemi Amoo,We celebrate the boy child today and every day...,NaN,NaN,NaN,NaN,NaN,NaN,6,6,6,0,NaN
8520,4/27/2023 8:18 pm,Instagram,Photo,Sophia Amuka,Make your money bloom with the right investmen...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,12/24/2014 5:37 pm,Instagram,Photo,,"Withdraw cash up to 5 times, pay bills or tran...",NaN,NaN,NaN,NaN,NaN,NaN,6,6,6,0,NaN
9996,12/24/2014 10:28 am,Instagram,Photo,,#CardDiaries week 13\n\nI love Christmas! In m...,NaN,NaN,NaN,NaN,NaN,NaN,6,6,6,0,NaN
9997,12/23/2014 5:14 pm,Instagram,Photo,,Eat-in this holidays. \nInstead of going out t...,NaN,NaN,NaN,NaN,NaN,NaN,6,6,6,0,NaN
9998,12/23/2014 10:38 am,Instagram,Photo,,"If you do not move forward, you will always be...",NaN,NaN,NaN,NaN,NaN,NaN,7,7,7,0,NaN


There seems to be a batch of posts having a key metric 'Reach' as nan or zero 
for the sake of the analysis we are considering posts with atleast reach of 1

In [32]:
#dropping rows where reach is NaN or zero
instagram_data = instagram_data.dropna(subset=['Reach'])
instagram_data = instagram_data[instagram_data['Reach'] != 0]
# Reset the index of the DataFrame
instagram_data.reset_index(drop=True, inplace=True)
instagram_data.isnull().sum()

Date                                  0
Network                               0
Content Type                          0
Sent by                               0
Post                                  0
Impressions                           0
Organic Impressions                   0
Reach                                 0
Organic Reach                         0
Engagement Rate (per Impression)     23
Engagement Rate (per Reach)         939
Engagements                           0
Reactions                             0
Likes                                 0
Comments                              0
Saves                                 0
dtype: int64

In [33]:
#split Date and Time from the date column
instagram_data['Datetime'] = pd.to_datetime(instagram_data['Date'], format = '%m/%d/%Y %I:%M %p')
instagram_data['Date only'] = instagram_data['Datetime'].dt.date
instagram_data['Time'] = instagram_data['Datetime'].dt.time
instagram_data[['Date only', 'Time']]


,Date only,Time
0,2023-01-30,17:06:00
1,2022-04-15,09:04:00
2,2021-05-28,20:45:00
3,2023-01-04,16:08:00
4,2021-12-07,18:52:00
...,...,...
8178,2017-01-25,10:43:00
8179,2017-01-24,17:44:00
8180,2017-01-24,11:07:00
8181,2017-01-23,17:25:00


In [34]:
instagram_data.head(1)

,Date,Network,Content Type,Sent by,Post,Impressions,Organic Impressions,Reach,Organic Reach,Engagement Rate (per Impression),Engagement Rate (per Reach),Engagements,Reactions,Likes,Comments,Saves,Datetime,Date only,Time
0,1/30/2023 5:06 pm,Instagram,Carousel,,The new Naira banknotes in circulation have se...,"39,681","39,681","32,418","32,418",4.89%,5.99%,"1,942","1,261","1,261",56,625.0,2023-01-30 17:06:00,2023-01-30,17:06:00


In [35]:
#dropping the date and datetime columns
instagram_data = instagram_data.drop(columns = ['Date', 'Datetime'])
#renaming the date only column to date
instagram_data = instagram_data.rename(columns = {'Date only' : 'Date'})
instagram_data.head(1)

,Network,Content Type,Sent by,Post,Impressions,Organic Impressions,Reach,Organic Reach,Engagement Rate (per Impression),Engagement Rate (per Reach),Engagements,Reactions,Likes,Comments,Saves,Date,Time
0,Instagram,Carousel,,The new Naira banknotes in circulation have se...,"39,681","39,681","32,418","32,418",4.89%,5.99%,"1,942","1,261","1,261",56,625.0,2023-01-30,17:06:00


In [36]:
#saving the file as a csv
instagram_data.to_csv('instagram_data.csv', index = False)


cleaning twitter data


In [37]:
#loading twitter data
twitter_data = pd.read_csv('Performance_Twitter.csv')
twitter_data.head()

,Date,Post ID,Network,Post Type,Content Type,Profile,Sent by,Link,Post,Linked Content,...,Video Removed from Playlists,Annotation Impressions,Annotation Clickable Impressions,Annotation Closable Impressions,Annotation Closes,Card Impressions,Card Teaser Impressions,Card Teaser Clicks,Poll Votes,Tags
0,7/13/2023 10:57 am,1679429829567946759,Twitter,Tweet,Photo,'@StanbicIBTC,Sophia Amuka,https://twitter.com/StanbicIBTC/status/1679429...,Your money should not be a gamble. Before you ...,https://twitter.com/StanbicIBTC/status/1679429...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CIB Posts
1,7/12/2023 8:03 pm,1679204777081864192,Twitter,Tweet,Text,'@StanbicIBTC,Blessing Ubah,https://twitter.com/StanbicIBTC/status/1679204...,"What skill (s) do you have, that no one ever t...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7/12/2023 7:00 pm,1679189012798660608,Twitter,Tweet,Photo,'@StanbicIBTC,Blessing Ubah,https://twitter.com/StanbicIBTC/status/1679189...,Sade was able to acquire a home loan and make ...,"https://bit.ly/StanbicIBTCHomeLoans, https://t...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Weekly Wealth Posts
3,7/12/2023 6:20 pm,1679179017084354560,Twitter,Tweet,Photo,'@StanbicIBTC,Blessing Ubah,https://twitter.com/StanbicIBTC/status/1679179...,Here are some mistakes you shouldn’t make duri...,https://twitter.com/StanbicIBTC/status/1679179...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7/11/2023 12:50 pm,1678733396992745472,Twitter,Tweet,Photo,'@StanbicIBTC,Sophia Amuka,https://twitter.com/StanbicIBTC/status/1678733...,Safeguard every aspect of your operations with...,https://twitter.com/StanbicIBTC/status/1678733...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Insurance Limited Posts, Weekly Wealth Posts"


In [38]:
#establish shape of the data and check for null values
print(twitter_data.shape)
print(twitter_data.isnull().sum())

(8529, 147)
Date                          0
Post ID                       0
Network                       0
Post Type                     0
Content Type                  0
                           ... 
Card Impressions           8529
Card Teaser Impressions    8529
Card Teaser Clicks         8529
Poll Votes                 8529
Tags                       7129
Length: 147, dtype: int64


In [39]:
#deleting columns with null values only
null_counts = twitter_data.isnull().sum()
threshold = 8529
columns_to_drop = null_counts[null_counts == threshold].index
twitter_data = twitter_data.drop(columns = columns_to_drop)
twitter_data.isnull().sum()

Date                                   0
Post ID                                0
Network                                0
Post Type                              0
Content Type                           0
Profile                                0
Sent by                                0
Link                                  48
Post                                   0
Linked Content                      3551
Impressions                          687
Organic Impressions                  687
Potential Reach                       80
Engagement Rate (per Impression)     688
Engagements                          687
Reactions                            687
Likes                                687
Comments                             687
Shares                               687
Click-Through Rate                  4916
Post Link Clicks                    4916
Other Post Clicks                    687
Post Clicks (All)                    687
Post Media Clicks                   1836
Post Hashtag Cli

In [40]:
#dropping columns with null values above 7000
null_counts =  twitter_data.isnull().sum()
threshold = 7000
columns_to_drop = null_counts[null_counts >= 7000].index
twitter_data = twitter_data.drop(columns = columns_to_drop)
twitter_data.isnull().sum()

Date                                   0
Post ID                                0
Network                                0
Post Type                              0
Content Type                           0
Profile                                0
Sent by                                0
Link                                  48
Post                                   0
Linked Content                      3551
Impressions                          687
Organic Impressions                  687
Potential Reach                       80
Engagement Rate (per Impression)     688
Engagements                          687
Reactions                            687
Likes                                687
Comments                             687
Shares                               687
Click-Through Rate                  4916
Post Link Clicks                    4916
Other Post Clicks                    687
Post Clicks (All)                    687
Post Media Clicks                   1836
Post Hashtag Cli

In [41]:
#dropping redundant columns and those not necessary to the analysis
redundant_columns = ['Post ID','Post Type', 'Profile', 'Link', 'Linked Content','Organic Impressions']
twitter_data = twitter_data.drop(columns = redundant_columns)
twitter_data.columns

Index(['Date', 'Network', 'Content Type', 'Sent by', 'Post', 'Impressions',
       'Potential Reach', 'Engagement Rate (per Impression)', 'Engagements',
       'Reactions', 'Likes', 'Comments', 'Shares', 'Click-Through Rate',
       'Post Link Clicks', 'Other Post Clicks', 'Post Clicks (All)',
       'Post Media Clicks', 'Post Hashtag Clicks', 'Post Detail Expand Clicks',
       'Profile Clicks', 'Other Engagements', 'App Engagements',
       'App Install Attempts', 'App Opens', 'Follows from Post',
       'Unfollows from Post', 'Media Views'],
      dtype='object')

In [42]:
#checking the remaining columns for varibility in data points
twitter_data.nunique()

Date                                8429
Network                                1
Content Type                           4
Sent by                               12
Post                                8509
Impressions                         4300
Potential Reach                     7857
Engagement Rate (per Impression)     927
Engagements                          619
Reactions                            210
Likes                                210
Comments                             124
Shares                               159
Click-Through Rate                   217
Post Link Clicks                      92
Other Post Clicks                    480
Post Clicks (All)                    492
Post Media Clicks                    364
Post Hashtag Clicks                   42
Post Detail Expand Clicks            257
Profile Clicks                        85
Other Engagements                    124
App Engagements                        3
App Install Attempts                   1
App Opens       

In [43]:
#scrutinizing variety of some columns of concern
print(twitter_data['App Engagements'].value_counts())
print(twitter_data['App Install Attempts'].value_counts())
print(twitter_data['App Opens'].value_counts())
print(twitter_data['Follows from Post'].value_counts())
print(twitter_data['Unfollows from Post'].value_counts())

App Engagements
0.0    7839
1.0       2
2.0       1
Name: count, dtype: int64
App Install Attempts
0.0    7842
Name: count, dtype: int64
App Opens
0.0    7839
1.0       2
2.0       1
Name: count, dtype: int64
Follows from Post
0.0     7702
1.0      114
2.0       13
3.0        5
6.0        3
8.0        1
15.0       1
5.0        1
21.0       1
4.0        1
Name: count, dtype: int64
Unfollows from Post
0.0    7664
1.0     166
2.0      11
3.0       1
Name: count, dtype: int64


In [44]:
#dropping those whose variability is not fitting to the analysis
twitter_data = twitter_data.drop(['App Engagements','App Install Attempts','App Opens'], axis =1)
twitter_data.isnull().sum()

Date                                   0
Network                                0
Content Type                           0
Sent by                                0
Post                                   0
Impressions                          687
Potential Reach                       80
Engagement Rate (per Impression)     688
Engagements                          687
Reactions                            687
Likes                                687
Comments                             687
Shares                               687
Click-Through Rate                  4916
Post Link Clicks                    4916
Other Post Clicks                    687
Post Clicks (All)                    687
Post Media Clicks                   1836
Post Hashtag Clicks                 2632
Post Detail Expand Clicks            687
Profile Clicks                       687
Other Engagements                    687
Follows from Post                    687
Unfollows from Post                  687
Media Views     

In [45]:
#for this analysis we are considering posts with atleast 1 impression
#dropping rows with zero or nul impressions
twitter_data = twitter_data.dropna(subset = ['Impressions'])
twitter_data = twitter_data[twitter_data['Impressions'] != 0]
twitter_data.isnull().sum()

Date                                   0
Network                                0
Content Type                           0
Sent by                                0
Post                                   0
Impressions                            0
Potential Reach                       69
Engagement Rate (per Impression)       1
Engagements                            0
Reactions                              0
Likes                                  0
Comments                               0
Shares                                 0
Click-Through Rate                  4229
Post Link Clicks                    4229
Other Post Clicks                      0
Post Clicks (All)                      0
Post Media Clicks                   1149
Post Hashtag Clicks                 1945
Post Detail Expand Clicks              0
Profile Clicks                         0
Other Engagements                      0
Follows from Post                      0
Unfollows from Post                    0
Media Views     

In [46]:
#dropping rows with a potential reach of zero and filling missing values with zero
twitter_data = twitter_data.dropna(subset = 'Potential Reach')
twitter_data = twitter_data.fillna(0)
twitter_data.isnull().sum()

Date                                0
Network                             0
Content Type                        0
Sent by                             0
Post                                0
Impressions                         0
Potential Reach                     0
Engagement Rate (per Impression)    0
Engagements                         0
Reactions                           0
Likes                               0
Comments                            0
Shares                              0
Click-Through Rate                  0
Post Link Clicks                    0
Other Post Clicks                   0
Post Clicks (All)                   0
Post Media Clicks                   0
Post Hashtag Clicks                 0
Post Detail Expand Clicks           0
Profile Clicks                      0
Other Engagements                   0
Follows from Post                   0
Unfollows from Post                 0
Media Views                         0
dtype: int64

In [47]:
#splitting date and time
twitter_data['Datetime'] = pd.to_datetime(twitter_data['Date'], format = '%m/%d/%Y %I:%M %p')
twitter_data['Date only']=twitter_data['Datetime'].dt.date
twitter_data['Time']=twitter_data['Datetime'].dt.time
twitter_data[['Date only', 'Time']]

,Date only,Time
0,2023-07-13,10:57:00
1,2023-07-12,20:03:00
2,2023-07-12,19:00:00
3,2023-07-12,18:20:00
4,2023-07-11,12:50:00
...,...,...
8413,2014-10-09,16:02:00
8414,2014-10-09,15:00:00
8415,2014-10-09,14:00:00
8416,2014-10-09,12:00:00


In [48]:
twitter_data.head(1)

,Date,Network,Content Type,Sent by,Post,Impressions,Potential Reach,Engagement Rate (per Impression),Engagements,Reactions,...,Post Hashtag Clicks,Post Detail Expand Clicks,Profile Clicks,Other Engagements,Follows from Post,Unfollows from Post,Media Views,Datetime,Date only,Time
0,7/13/2023 10:57 am,Twitter,Photo,Sophia Amuka,Your money should not be a gamble. Before you ...,445,"345,718",3.6%,16,4,...,0.0,7,1.0,0,0.0,0.0,1,2023-07-13 10:57:00,2023-07-13,10:57:00


In [49]:
#cleaning up the columns
twitter_data  =twitter_data.drop(['Date', 'Datetime'], axis =1)
twitter_data = twitter_data.rename(columns = {'Date only':'Date'})
twitter_data.head(1)

,Network,Content Type,Sent by,Post,Impressions,Potential Reach,Engagement Rate (per Impression),Engagements,Reactions,Likes,...,Post Media Clicks,Post Hashtag Clicks,Post Detail Expand Clicks,Profile Clicks,Other Engagements,Follows from Post,Unfollows from Post,Media Views,Date,Time
0,Twitter,Photo,Sophia Amuka,Your money should not be a gamble. Before you ...,445,"345,718",3.6%,16,4,4,...,1,0.0,7,1.0,0,0.0,0.0,1,2023-07-13,10:57:00


In [50]:
#saving to csv
twitter_data.to_csv('twitter_data.csv', index = False)


cleaning linkedin data


In [51]:
#loading the data
linkedin_data = pd.read_csv('Performance_LinkedIn.csv')
linkedin_data.head()

,Date,Post ID,Network,Post Type,Content Type,Profile,Sent by,Link,Post,Linked Content,...,Video Removed from Playlists,Annotation Impressions,Annotation Clickable Impressions,Annotation Closable Impressions,Annotation Closes,Card Impressions,Card Teaser Impressions,Card Teaser Clicks,Poll Votes,Tags
0,7/13/2023 12:03 pm,7085212016608456704,LinkedIn,Post,Photo,Stanbic IBTC,Blessing Ubah,https://linkedin.com/feed/update/urn:li:share:...,Your money should not be a gamble. Before you ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CIB Posts
1,7/12/2023 8:00 pm,7084969706025009152,LinkedIn,Post,Text,Stanbic IBTC,Blessing Ubah,https://linkedin.com/feed/update/urn:li:share:...,"What skill (s) do you have, that no one ever t...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7/12/2023 7:00 pm,7084954641788018688,LinkedIn,Post,Photo,Stanbic IBTC,Blessing Ubah,https://linkedin.com/feed/update/urn:li:ugcPos...,Sade was able to acquire a home loan and make ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Weekly Wealth Posts
3,7/11/2023 2:49 pm,7084529022105231360,LinkedIn,Post,Photo,Stanbic IBTC,Blessing Ubah,https://linkedin.com/feed/update/urn:li:share:...,Safeguard every aspect of your operations with...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Weekly Wealth Posts
4,7/11/2023 9:17 am,7084445462967070721,LinkedIn,Post,Photo,Stanbic IBTC,Blessing Ubah,https://linkedin.com/feed/update/urn:li:share:...,"Together, we can make a difference and build a...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
#determining shape of the data and null values
print(linkedin_data.shape)
linkedin_data.isnull().sum()

(7760, 147)


Date                          0
Post ID                       0
Network                       0
Post Type                     0
Content Type                  0
                           ... 
Card Impressions           7760
Card Teaser Impressions    7760
Card Teaser Clicks         7760
Poll Votes                 7758
Tags                       6120
Length: 147, dtype: int64

In [53]:
#dropping columns with more than 6000 null values
null_counts = linkedin_data.isnull().sum()
threshold = 6000
columns_to_drop = null_counts[null_counts >= threshold].index
linkedin_data = linkedin_data.drop(columns = columns_to_drop)
linkedin_data.isnull().sum()

Date                                   0
Post ID                                0
Network                                0
Post Type                              0
Content Type                           0
Profile                                0
Sent by                                0
Link                                   4
Post                                  31
Impressions                         1428
Organic Impressions                 1428
Engagement Rate (per Impression)    1428
Engagements                         1428
Reactions                           1428
Likes                               1428
Comments                            1428
Shares                              1428
Click-Through Rate                  1428
Post Link Clicks                    1428
Post Clicks (All)                   1428
dtype: int64

For the analysis we are considering posts with at least one impression

In [54]:
#dropping rows with null or zero impressions
linkedin_data = linkedin_data.dropna(subset = ['Impressions'])
linkedin_data = linkedin_data[linkedin_data['Impressions'] != 0]
linkedin_data.isnull().sum()

Date                                 0
Post ID                              0
Network                              0
Post Type                            0
Content Type                         0
Profile                              0
Sent by                              0
Link                                 0
Post                                31
Impressions                          0
Organic Impressions                  0
Engagement Rate (per Impression)     0
Engagements                          0
Reactions                            0
Likes                                0
Comments                             0
Shares                               0
Click-Through Rate                   0
Post Link Clicks                     0
Post Clicks (All)                    0
dtype: int64

In [55]:
#dropping rows with null posts and other redundant columns
linkedin_data = linkedin_data.dropna(subset = ['Post'])
linkedin_data = linkedin_data.drop(['Post ID','Post Type','Profile','Link','Organic Impressions'], axis = 1)
linkedin_data.isnull().sum()

Date                                0
Network                             0
Content Type                        0
Sent by                             0
Post                                0
Impressions                         0
Engagement Rate (per Impression)    0
Engagements                         0
Reactions                           0
Likes                               0
Comments                            0
Shares                              0
Click-Through Rate                  0
Post Link Clicks                    0
Post Clicks (All)                   0
dtype: int64

In [56]:
#split date and time
linkedin_data['Datetime'] = pd.to_datetime(linkedin_data['Date'], format = '%m/%d/%Y %I:%M %p')
linkedin_data['Date only'] = linkedin_data['Datetime'].dt.date
linkedin_data['Time'] = linkedin_data['Datetime'].dt.time
linkedin_data[['Date only', 'Time']]

,Date only,Time
2,2023-07-12,19:00:00
3,2023-07-11,14:49:00
4,2023-07-11,09:17:00
5,2023-07-10,18:04:00
6,2023-07-10,16:21:00
...,...,...
6666,2018-01-31,15:30:00
6669,2018-01-30,14:00:00
6670,2018-01-30,10:00:00
6671,2018-01-29,12:12:00


In [57]:
linkedin_data.head(1)

,Date,Network,Content Type,Sent by,Post,Impressions,Engagement Rate (per Impression),Engagements,Reactions,Likes,Comments,Shares,Click-Through Rate,Post Link Clicks,Post Clicks (All),Datetime,Date only,Time
2,7/12/2023 7:00 pm,LinkedIn,Photo,Blessing Ubah,Sade was able to acquire a home loan and make ...,63,9.52%,6,3.0,3.0,0.0,0.0,4.76%,3,3,2023-07-12 19:00:00,2023-07-12,19:00:00


In [58]:
#cleaning up the columns
linkedin_data = linkedin_data.drop(['Date', 'Datetime'], axis =1)
linkedin_data = linkedin_data.rename(columns = {'Date only':'Date'}) 
linkedin_data.head(1)

,Network,Content Type,Sent by,Post,Impressions,Engagement Rate (per Impression),Engagements,Reactions,Likes,Comments,Shares,Click-Through Rate,Post Link Clicks,Post Clicks (All),Date,Time
2,LinkedIn,Photo,Blessing Ubah,Sade was able to acquire a home loan and make ...,63,9.52%,6,3.0,3.0,0.0,0.0,4.76%,3,3,2023-07-12,19:00:00


In [59]:
#saving data to csv
linkedin_data.to_csv('linkedin_data.csv',index = False)